In [17]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.upper_layer1 = nn.Sequential(
            nn.Conv2d(4,64,7),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.upper_layer2 = nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.under_layer1 = nn.Sequential(
            nn.Conv2d(4,160,13),
            nn.ReLU()
        )
        self.final_layer = nn.Conv2d(224,5,21)
        #self.upper_conv1 = nn.Conv2d(4,64,7)
        #self.upper_conv2 = nn.Conv2d(64,64,3)
        #self.under_conv1 = nn.Conv2d(4,160,13)
        #self.final_conv = nn.Conv2d(224,5,21)
    
    def forward(self, x):
        upper_x = self.upper_layer2(self.upper_layer1(x))
        under_x = self.under_layer1(x)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv1(x)), (4, 4),stride = 1)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv2(upper_x)), (2, 2), stride = 1)
        #under_x = F.relu(self.under_conv1(x))
        final_x = torch.cat((under_x, upper_x), 1)
        out = F.softmax(self.final_layer(final_x))
        #out = out.view(-1,5)
        return out
        
net = TwoPathConv()
print(net)

x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)
#y_pred = y_pred.data.resize_(1,5)
#y_pred = Variable(y_pred,requires_grad = True)
#print(y_pred.size())
#print(y_pred.view)

TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)


In [3]:
f=open("trainval-balanced.txt", "r")
content=f.readlines()
data=[]
data_train=[]
data_val=[]
i=0
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    data.append([])
    data[i].append(item[0])
    data[i].append(int(item[1]))
    data[i].append(int(item[2]))
    data[i].append(int(item[3]))
    data[i].append(int(item[4]))
    if i%10000==0:
        data_val.append(data[i])
    else:
        data_train.append(data[i])
    i += 1
f.close()
import h5py
f = h5py.File('training.h5','r')

In [18]:
print len(data)
print len(data_val)
print len(data_train)

8998296
900
8997396


In [5]:
print torch.cuda.is_available()

True


In [6]:
def create_batch(batch_mask):
    X_batch = []
    y_batch = []
    for i in range(len(batch_mask)):
        case, x, y, z, l = data_train[batch_mask[i]]
        case1 = case[:2]
        case2 = case[3:]
        X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_batch.append(l)
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

def create_val(batch_mask):
    X_val=[]
    y_val=[]
    for i in range(len(batch_mask)):
        case, x, y, z, l = data_val[batch_mask[i]]
        case1 = case[:2]
        case2 = case[3:]
        X_val.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_val.append(l)
    X_val = torch.from_numpy(np.array(X_val))
    y_val = torch.from_numpy(np.array(y_val))
    return X_val, y_val

In [24]:
import time
import numpy as np
num_train=len(data_train)
num_val=len(data_val)
batch_size=900
val_size=num_val
num_epoch=10.0
num_times=int(float(num_train)/batch_size*num_epoch)
print num_times
learning_rate = 1e-4
reg=1e-10
net = torch.load("tumorseg.pkl")
net.cuda()
prev_time = time.clock()
for i in xrange(100):  # loop over the dataset multiple times
    batch_mask = np.random.choice(num_train, batch_size)
    X_batch=None
    X_batch, y_batch = create_batch(batch_mask)
    X_batch, y_batch = Variable(X_batch.cuda()), Variable(y_batch.cuda(), requires_grad = False)
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=reg)
    optimizer.zero_grad()
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
        print loss
        print 'time used %.3f' % (time.clock()-prev_time)
        val_mask=np.random.choice(num_val, val_size)
        X_val, y_val=create_val(val_mask)
        X_val=Variable(X_val.cuda(), requires_grad=False)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda()).sum()
        print('Validation accuracy:', float(correct)/val_size)


99971
Variable containing:
 1.2326
[torch.cuda.FloatTensor of size 1 (GPU 0)]

time used 2.041
('Validation accuracy:', 0.7233333333333334)
Variable containing:
 1.1938
[torch.cuda.FloatTensor of size 1 (GPU 0)]

time used 25.018
('Validation accuracy:', 0.6788888888888889)
Variable containing:
 1.2157
[torch.cuda.FloatTensor of size 1 (GPU 0)]

time used 48.210
('Validation accuracy:', 0.71)
Variable containing:
 1.2389
[torch.cuda.FloatTensor of size 1 (GPU 0)]

time used 71.251
('Validation accuracy:', 0.7144444444444444)
Variable containing:
 1.2138
[torch.cuda.FloatTensor of size 1 (GPU 0)]

time used 94.257
('Validation accuracy:', 0.7188888888888889)
Variable containing:
 1.2085
[torch.cuda.FloatTensor of size 1 (GPU 0)]

time used 117.390
('Validation accuracy:', 0.7011111111111111)
Variable containing:
 1.1884
[torch.cuda.FloatTensor of size 1 (GPU 0)]

time used 140.512
('Validation accuracy:', 0.7288888888888889)
Variable containing:
 1.1763
[torch.cuda.FloatTensor of size 1

In [26]:
torch.save(net, "tumorseg.pkl")

In [192]:
import SimpleITK
input_image = SimpleITK.ReadImage('Test1.mha')
image_2D = SimpleITK.Image(64, 64, SimpleITK.sitkFloat32)
print(input_image.GetSize())
#help(image_2D)

(158, 180, 140)


In [127]:

from graphviz import Digraph
import re
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Variable
import torchvision.models as models


def make_dot(var):
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def add_nodes(var):
        if var not in seen:
            if isinstance(var, Variable):
                value = '('+(', ').join(['%d'% v for v in var.size()])+')'
                dot.node(str(id(var)), str(value), fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'previous_functions'):
                for u in var.previous_functions:
                    dot.edge(str(id(u[0])), str(id(var)))
                    add_nodes(u[0])
    add_nodes(var.creator)
    return dot


inputs = torch.randn(1,3,224,224)
resnet18 = models.resnet18()
y = resnet18(Variable(inputs))
# print(y)
x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)

g = make_dot(y_pred)
g.view()
print(g)

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'Digraph.gv'], make sure the Graphviz executables are on your systems' PATH